## **Transforming, Grouping & Sorting Data**
Using functions such as `map()` and `apply()` enable us to transform our data. These functions return a transformed data without modifying the original data.  

### **Learning outcome:**
- To learn how to transform our data (series or dataframe) using `map()` and `apply()` for mapping values
- Use lambda for column and row operations when we use mapping function.
- Introduce methods to aggregate data points using `groupby()` and working with Multiindex

## Data transformation.
To map our data, is to use a function that enables us to take one set of values and "map" them to another set of values. Why would one do this? In data science, we often have to create a new representation from the exisiting data, or we need to transform our data from the current format to a different format to do downstream analysis. In this notebook we will introduce two mapping methods; `map()` and `apply()`

&nbsp;
####  **Crime Rates Dataset**
We will continue to use the crime rate dataset from M2_CL5

In [ ]:
# Importing libraries
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

In [1]:
## Read in data set and converting it to a pandas dataframe
crime_rates = pd.read_csv('https://raw.githubusercontent.com/csbfx/advpy122-data/master/crime_rates.csv',
                          names=['state', 'year', 'pop', 'violent', 'murder', 'rape', 'robbery', 'assault', 'property', 'burglary', 'larceny','vehicle'],
                          header=0
                          )

## A reminder of what columns are in this dataset and the size of the dataframe
crime_rates.info()

In [2]:
## To show the statistic of our int columns
crime_rates.describe()

##  Dropping columns with `drop()`
This data set as we previously observed is clean and does not have redundant data. For practice, we will assume that we do not need the column "year" or "pop".  Pandas DataFrame method `drop()` will return a new DataFrame by default but it will not overwrite the current DataFrame. If you want to overwrite the current DataFrame, you can set the argument `inplace=True`.  

&nbsp;
The drop() method can drop either rows or columns. The default is rows (`axis=0`). To drop columns, set the argument to `axis=1`.

In [3]:
crime_rates_dropped = crime_rates.drop(['year','pop'], axis=1)
crime_rates_dropped.head()

## `map()` ##
The Series method [`map()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html) maps values of Series according to an input function and returns a new Series.   

> `new_series = a_series.map(some_function)`

**Scenario**: Suppose that we wanted to redefine states to broad region using a mapping function.

We will be using a <i>**lambda function**</i> in the example below. You can review Python lambda function [here](https://www.w3schools.com/python/python_lambda.asp).

In [5]:
df = pd.DataFrame({'Genre': ['Action', 'Comedy', 'Drama', 'Horror']})
genre_map = {'Action':'A', 'Comedy':'C', 'Drama':'D', 'Horror':'H'}

### map()
df['S_Genre'] = df['Genre'].map(genre_map)

### apply() on a series
df['Uppercase_Genre'] = df['Genre'].apply(lambda x: x.upper())
df

In [6]:
## Example function that adds 5 to argument passed in
def fxn_add5(x):
    return x+5
print(fxn_add5(10))

## Rewrite the function above as a lambda function
## lambda arg: expression
arg_add5 = lambda x: x+5
list(map(arg_add5, [10, 9, 8]))# Using map to apply the lambda function to each element of the list

###  Mapping values with `map()`
Here we will map states to broad regions using a mapping function.

In [7]:
## Use a dictionary to define the regions
region_map = {
    'California': 'West',
    'Washington': 'West',
    'New York': 'Northeast',
    'Illinois': 'Midwest',
    'Colorado': 'Midwest',
    'Alabama': 'South',
    'Arkansas': 'South',
    'Arizona': 'South',
    'Texas': 'South',
    'Georgia': 'South',
    'Florida': 'South'
}
crime_rates['Region'] = crime_rates['state'].map(lambda x: region_map.get(x, 'Other'))
crime_rates[['state','Region']].drop_duplicates().head() #This does not overwrite the DataFrame

The function you pass to `map()` should expect a single value from the Series (a point value, in the above example), and return a transformed version of that value. `map()` returns a new Series where all the values have been transformed by your function.

Create a new column called 'pop-scale', which stores the values we define the scale of a population depending on the size of the population. We know from using
> crime_rates.describe()  

that the population ranges from 226,167.00 to 35,484,453.00
-  \> 20,000,001.00 - 3 (large population)
-  between 1,000,001.00 and 20,000,000.00 - 2 (midsize population)
-  below 1,000,000.00 - 1 (small population)

In [8]:
## Create function to do transformation for map()
# Since we are only looking at population(a single column), we do not need to use apply()

def pop_scale(x): # x is going to be a value from the Series
    if x > 20000001:
        return 3
    elif 1000001 < x <= 20000000:
        return 2
    else:
        return 1

scale = crime_rates['pop'].map(pop_scale) # should return a Series that contains the star values
crime_rates['pop_scale'] = scale # Add a new column scale to the DataFrame wine
crime_rates.head()

##  Applying transformations with `apply()`
The DataFrame method, [`apply()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html), should be used if we want to <u>transform the whole DataFrame</u> by calling a custom method on each row. We can use `apply()` to pass a function and apply it on every single value of the panda series.  

Below we will call `crime_rates.apply()` with `axis='columns'`. If we use `axis='index'`, then instead of passing a function to transform each row for the column(s), we would need to provide a function to transform each column for the row(s).

Recall:  
- **Single column:** similar to `.map()`.
- **Row-wise operations:** use `axis='columns'` or `axis=1`.

In [9]:
## We are going to define if a Crime level is consider "high" based on the rate of violent crimes
crime_rates['crime_level'] = crime_rates['violent'].apply(
    lambda x: 'High' if x > 1000 else ('Med' if 999 > x > 500 else 'Low')
)
crime_rates[['violent','crime_level']].head()

In [10]:
## Combine multiple types of crime to get the total crime rate
crime_rates['total_crime'] = crime_rates.apply(
    lambda row: row['violent'] + row['murder'] + row['assault'],
    axis='columns'
)
crime_rates[['violent','murder', 'assault', 'total_crime']].head()

When we are performing an operation between a lot of values, pandas looks at the expression and figures out that we must mean to do the mathematical expression on in the dataset.

Pandas will also understand what to do if we perform these operations between Series of equal length. For example, we combine the values of different crime type in the dataset. You can also combine strings, but you can not combine int and string values.

**Note:**
`map()` and `apply()` **return new, transformed Series and DataFrames, respectively**. They <i>**don't modify**</i> the original data they're called on.

# Groupwise analysis
Any time we see a question involving the words ”how many ... for each ...” the answer is `value_counts`. We can replicate what `value_counts()` does by doing the following:

In [11]:
## How many states have the high level of crime?
crime_rates.groupby('crime_level').state.count()

In [12]:
crime_rates['crime_level'].value_counts()

[`groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) created different category of crime level based on the number of violet crime rates to a given state. For each of these groups, we grabbed the `crime_level()` column and counted how many times it appeared. value_counts() is just a shortcut to this `groupby()` operation.

We can use any of the summary functions with this data. For example, to get the lowest violent crime rate for each year, we can do the following:

In [13]:
crime_rates.groupby('year').violent.min().head()

In [14]:
## To get the state that is associated with the groupby, we can reference the index
min_violent_by_year_index = crime_rates.groupby('year')['violent'].idxmin()

## Use .loc to pull out the rows that contain the index of interest
state_with_min_violent = crime_rates.loc[min_violent_by_year_index, ['year', 'state', 'violent']]
state_with_min_violent.head()

You can think of each group we generate as being a slice of our DataFrame containing only data with values that match. This DataFrame is accessible to us directly using the `apply()` method, and we can then manipulate the data in any way we see fit.

##  Aggregation with `groupby()`
Another `groupby()` method worth mentioning is `agg()`, which lets you run a bunch of different functions on your DataFrame simultaneously. For example, we can generate a simple statistical summary of the dataset as follows:

We will group by `year`, and compute mean values for the different crime types across the states.

In [15]:
grouped = crime_rates.groupby(['year'])[['violent','murder', 'assault']].mean()
grouped.head()

## Multi-Index

In all of the examples we've seen thus far we've been working with DataFrame or Series objects with a single-label index. `groupby()` is slightly different in the fact that, depending on the operation we run, it will sometimes result in what is called a multi-index.

A multi-index differs from a regular index in that it has multiple levels. Multi-indices have several methods for dealing with their tiered structure which are absent for single-level indices. They also require two levels of labels to retrieve a value. Dealing with multi-index output is a common "gotcha" for users new to pandas.

The use cases for a multi-index are detailed alongside instructions on using them in the [MultiIndex / Advanced Selection](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html) section of the pandas documentation.

However, in general the multi-index method you will use most often is the one for converting back to a regular index, the `reset_index()` method:

In [ ]:
grouped_flat = grouped.reset_index()
grouped_flat.head()

# Sorting
Looking again at `state_with_min_violent` we can see that grouping returns data in index order, not in value order. That is to say, when outputting the result of a `groupby`, the order of the rows is dependent on the values in the index, not in the data.

To get data in the order want it in we can sort it ourselves. The `sort_values()` method is handy for this. [`sort_values()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) defaults to an ascending sort, where the lowest values go first. However, most of the time we want a descending sort, where the higher numbers go first.

##  Sorting with `sort_values()`

In [16]:
sorted_df = state_with_min_violent.sort_values(
    by=['year','state', 'violent'],
    ascending=[True, False, False]
)
sorted_df.head()

##  Class Exercise

### Class Exercise CL6.1
Identify the top three states per year by "property" crime rates.

### Class Exercise CL6.2
**Normalize Violent Crime Rate**

Compute a new column, violent_rate_per_100k, that represents violent incidents per 100,000 people:

violent_rate_per_100k = violent / pop * 100000

Use `apply()` either column-wise or row-wise to perform the calculation.

### Class Exercise CL6.3
**Understanding total crime rates**  
Group by `year` and `state`, sum the rate of `violent`, `property`, `burgalry`, `larceny`, `vehicle` crimes into `crime_total`. Sort the DataFrame by `year` and `crime_total`.

### Class Exercise CL6.4
**Crime Change by State year over year**  
Use `groupby()` and `shift()` to compute the percentage change in violent crime compared to the previous year for each state. You will want to group by state and sort_values() to display your data chronologically.  
note: `shift()` removes the first element from the array and returns that removed element.